In [ ]:
!pip install langchain langchain-community langchain-pinecone langchain-openai pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.3/46.3 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.6/587.6 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.0/240.0 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.2/52.2 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.1/221.1 kB 13.3 MB/s eta 0:00:00


# api 받아오기

In [12]:
import requests
from pprint import pprint
import time
from dotenv import load_dotenv
import os

end_point = 'https://apis.data.go.kr/1471000/HtfsInfoService03'

url = 'http://apis.data.go.kr/1471000/HtfsInfoService03/getHtfsItem01?'

load_dotenv()
decoding = os.getenv['DECOIDNG']

all_items = []  # 전체 데이터를 모을 리스트
page = 1        # 시작 페이지

while True:
    # 요청 변수 세팅
    params = {
        "pageNo": str(page),
        "numOfRows": "100",  # API 최대 허용값
        "ServiceKey": "decoding",
        "type": "json",
    }

    # API 요청
    response = requests.get(url, params=params, timeout=10)
    response.raise_for_status()

    # 결과 확인
    data = response.json()
    body = data.get('body')
    if not body:
        print(f"{page}페이지 - body 없음 종료")
        break

    items = body.get('items')
    if not items:
        print(f"{page}페이지 - items 없음 종료")
        break

    all_items.extend(items)
    print(f"{page}페이지 수집 완료 - 누적 {len(all_items)}개")

    page += 1


#수집된 데이터를 파일로 저장
with open('raw_data.json', 'w', encoding='utf-8') as f:
    json.dump(all_items, f, ensure_ascii=False, indent=4)

print(f"총 {len(all_items)}개 데이터 저장 완료")



# num_of_rows = 100
# page = 1

# # 3) 요청 변수(params) 세팅
# params = {
#     "pageNo":    "1",          # 페이지 번호
#     "numOfRows": str(num_of_rows),         # 한 페이지 결과 수
#     "ServiceKey": 'nsfVX4dKQRFTeyldmuRefFQqL8xOsDkkyw8TsU4dA4fO9vq7Zl7JTbrakHnVYBqRG62CWBhhOVwBaGgCBbm3AA==', # Swagger에 표시된 정확한 이름(ServiceKey)
#     "type":      "json",       # 응답 포맷(xml/json) – default: xml
#     # (필요시 API별 추가 파라미터를 여기에 더합니다)
# }

# # 4) 실제 요청 보내기
# response = requests.get(url, params=params, timeout=10)
# response.raise_for_status()

# # 5) 결과 확인
# pprint(response.json())

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
def extract_product_info(api_response):
    items = api_response['body']['items']

    simplified_products = []

    for entry in items:
        item = entry['item']
        simplified_products.append({
            '제품명': item.get('PRDUCT', '').strip(),
            '기능성': item.get('MAIN_FNCTN', '').strip(),
            '제조사': item.get('ENTRPS', '').strip(),
            '섭취방법': item.get('SRV_USE', '').strip(),
            '보관방법': item.get('PRSRV_PD', '').strip(),
            '주의사항': item.get('INTAKE_HINT1', '').strip(),
        })

    return simplified_products

product_list = extract_product_info(response.json())

# 출력문 (예쁘게 정리)
for i, p in enumerate(product_list[:3], 1):  # TOP3만 예시
    print(f"[{i}] {p['제품명']}")
    print(f"기능성: {p['기능성']}")
    print(f"제조사: {p['제조사']}")
    print(f"섭취방법: {p['섭취방법']}")
    print(f"보관방법: {p['보관방법']}")
    print(f"주의사항: {p['주의사항']}")
    print("-" * 50)

[1] 11종 혼합유산균
기능성: 유산균 증식 및 유해균 억제·배변활동 원활에 도움을 줄 수 있음
제조사: 일동바이오사이언스(주)
섭취방법: 건강기능식품 원료로 사용
보관방법: 냉장조건
주의사항: 1. 질환이 있거나 의약품 복용 시 전문가와 상담하십시오.
2. 알레르기 체질 등은 개인에 따라 과민반응을 나타낼 수 있습니다.
3. 어린이가 함부로 섭취하지 않도록 일일섭취량 방법을 지도해 주십시오.
4. 이상사례 발생 시 섭취를 중단하고 전문가와 상담하십시오.
--------------------------------------------------
[2] 6년근 고려홍삼정 PREMIUM
기능성: ①면역력 증진②피로개선③혈소판 응집 억제를 통한 혈액흐름에 도움④기억력 개선⑤항산화에 도움을 줄 수 있음
제조사: 주식회사 성윤 에프엔지(F&G)
섭취방법: 1일 3회, 1회 1스푼(3.2g)씩  물에 녹여 섭취 하십시요.
보관방법: 직사광선을 피해 건조하고 서늘한 곳에서 보관한다.
주의사항: [홍삼제품]의약품(당뇨치료제, 혈액항응고제) 복용 시 섭취에 주의
2) 특이체질등 알레르기 체질의 경우 제품성분을 확인 후 섭취하시기 바랍니다.
 3) 15세 이하의 어린이는 상기 섭취량의 절반 정도를 섭취하시요.
 4) 제품 개봉 또는 섭취시에 포장재로 인한 상처를 입을수 있으니주의 하십시오.
--------------------------------------------------
[3] 6년근 홍삼 자신만만 홍삼스틱
기능성: ⑴ 면역력 증진에 도움을 줄 수 있음
  ⑵ 피로개선에 도움을 줄 수 있음
  ⑶ 혈소판 응집 억제를 통한 혈액흐름에 도움을 줄 수 있음
  ⑷ 기억력 개선에 도움을 줄 수 있음
  ⑸ 항산화에 도움을 줄 수 있음
제조사: (주)일화
섭취방법: 1일 1회, 1회 1포를 섭취하십시오.
보관방법: 습기와 직사광선을 피하여 실온에 보관하시고, 개봉 후에는 반드시 냉장 보관하십시오.
주의사항: ⑴ 특수체질이거나 알레르기 체질인 경우에는 섭취 전에 반드시 원료명 

In [10]:
print(len(product_list))

10


# document생성

In [6]:
from langchain.schema import Document

def convert_to_documents(product_list):
    docs = []

    for product in product_list:
        content = (
            f"제품명: {product['제품명']}\n"
            f"기능성: {product['기능성']}\n"
            f"섭취방법: {product['섭취방법']}\n"
            f"보관방법: {product['보관방법']}\n"
            f"주의사항: {product['주의사항']}"
        )

        metadata = {
            "제품명": product["제품명"], # 중복으로 넣음
            "제조사": product["제조사"],
        }

        docs.append(Document(page_content=content, metadata=metadata))

    return docs

documents = convert_to_documents(product_list)

# 임베딩

In [8]:
from pinecone import Pinecone
from langchain_openai import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore

pc = Pinecone(api_key=os.environ['PINECONE_API_KEY'])

# 임베딩 모델
embeddings = OpenAIEmbeddings(model=os.environ['OPENAI_EMBEDDING_MODEL'])

# 벡터스토어 객체(client) 생성
vector_store = PineconeVectorStore.from_documents(
    documents,
    embedding=embeddings,
    index_name='healthcare'
)

# Pinecone 저장

In [ ]:
from langchain.vectorstore import pinecone



# 쿼리 날려